In [2]:
%classpath add mvn com.cra.figaro figaro_2.11 5.0.0.0

In [3]:
import com.cra.figaro.language._
import com.cra.figaro.algorithm.sampling._
import com.cra.figaro.library.compound.If
import math._

import com.cra.figaro.language._
import com.cra.figaro.algorithm.sampling._
import com.cra.figaro.library.compound.If
import math._


In [4]:
val hw = Constant("Oh hai there!")
// Create an algorithm that samples the string
val alg = Importance(1000, hw)
alg.start()

null

In [5]:
println(alg.probability(hw, "Oh hai there!"))
println(alg.probability(hw, "Oh hey there!"))

1.0
0.0


## Representation

In [6]:
Constant(6)
Constant("Hello") // every Figaro element has a value type

Constant(Hello)

All figaro elements are instances of the Element class.

***Atomic*** elements: don't depend on any other elements

In [6]:
Flip(0.7) // Element[Boolean]

Flip(0.7)

***Compound*** elements depend on other Elements

In [7]:
import com.cra.figaro.library.atomic.continuous._

Flip(Uniform(0.0, 1.0)) // argument is Element[Double]

If(Flip(0.7), Constant(1),
  Select(0.4 -> 2, 0.6 -> 3))

If(Flip(0.7), Constant(1), Select(0.4 -> 2, 0.6 -> 3))

### Building Compound Elements using *Chain*

In [8]:
Chain(Flip(0.7), (b: Boolean) =>
     if (b) Constant(1); else Select(0.4 -> 2, 0.6 -> 3))

// this represents the same model as above

Chain(Flip(0.7), <function1>)

### Apply

"Lifts" Scala functions

In [9]:
Apply(Select(0.2 -> 1, 0.8 -> 2), (i: Int) => i + 5)
// the function on the right is applied to the output
// of the expression on the left

Apply(Select(0.2 -> 1, 0.8 -> 2), <function1>)

In [10]:
Apply(Select(0.2 -> "a", 0.8 -> "b"), (i: String) => i + "c")

Apply(Select(0.2 -> a, 0.8 -> b), <function1>)

## Creating models

### Creating a Bayesian Network (using CPD)

In [25]:
import com.cra.figaro.language._
import com.cra.figaro.library.compound._ // includes CPD

val burglary = Flip(0.01) // "field", immutable
val earthquake = Flip(0.0001)
val alarm = CPD(burglary, earthquake,
               (false, false) -> Flip(0.001),
               (false, true) -> Flip(0.1),
               (true, false) -> Flip(0.9),
               (true, true) -> Flip(0.99))

val johnCalls = CPD(alarm,
                   false -> Flip(0.01),
                   true -> Flip(0.7))


Chain(Chain(Apply(Flip(0.01), Flip(1.0E-4), <function2>), <function1>), <function1>)

In [35]:
val alg = Importance(2000, johnCalls)
alg.start()
println(alg.probability(johnCalls, true))

0.016499999999999834


null

In a ***RichCPD***, each clause is a tuple of cases, one per parent.It can be OneOf a set, NoneOf (i.e. matches everything except those listed), or * , meaning anything. 

In [40]:
import com.cra.figaro.language._
import com.cra.figaro.library.compound._
val x1 = Select(0.1 -> 1, 0.2 -> 2, 0.3 -> 3, 0.4 -> 4)
val x2 = Flip(0.6) 
val x3 = Constant(5)
val x4 = Flip(0.8)
val y = RichCPD(x1, x2, x3, x4,
                (OneOf(1, 2), *, OneOf(5), *) -> Flip(0.1),
                (NoneOf(4), OneOf(false), *, *) -> Flip(0.7),
                (*, *, NoneOf(6, 7), OneOf(true)) -> Flip(0.9),
                (*, *, *, OneOf(false)) -> Constant(true))

Chain(Apply(Select(0.1 -> 1, 0.2 -> 2, 0.3 -> 3, 0.4 -> 4), Flip(0.6), Constant(5), Flip(0.8), <function4>), <function1>)

### Adding constraints and conditions to models

Every element has a condition, f(Element -> Boolean). 

Can be used to specify evidence or a structural property (e.g. only one of two teams can be a winner). The default condition is true for all values.

In [41]:
val x1 = Select(0.1 -> 1, 0.2 -> 2, 0.3 -> 3, 0.4 -> 4)

Select(0.1 -> 1, 0.2 -> 2, 0.3 -> 3, 0.4 -> 4)

In [43]:
x1.setCondition((i: Int) => i == 1 || i == 4) // i must be 1 or 4

In [47]:
// can add more conditions
x1.addCondition((i: Int) => i % 2 == 1)

In [51]:
x1.observe(2) // removes all previous conditions

### Constraint - weighting on an element. T => Double

Values should be non-negative. They should also be at most 1, otherwise *lazy factored inforence* algorithms will return errors (there'll be a warning). 

Constraints can be used for soft evidence. 

In [53]:
johnCalls.setConstraint((b: Boolean) => 
                       if (b) 1.0; else 0.1) // initially 0.01

In [54]:
val alg = Importance(2000, johnCalls)
alg.start()
println(alg.probability(johnCalls, true))

0.16588685665674197


null

In [55]:
import com.cra.figaro.library.atomic._
import com.cra.figaro.library.compound.If

import com.cra.figaro.library.atomic._
import com.cra.figaro.library.compound.If


In [56]:
class Firm {
    val efficient = Flip(0.3)
    val bid = If(efficient, continuous.Uniform(5, 15),
                continuous.Uniform(10, 20))
}

defined class Firm


In [57]:
val firms = Array.fill(20)(new Firm) // nice
val winner = discrete.Uniform(firms:_*) // "unpacks" array field
val winningBid = Chain(winner, (f: Firm) => f.bid)
winningBid.setConstraint((d: Double) => 20 - d)

null

In [79]:
class Person {
    val smokes = Flip(0.6)
}

val alice, bob, clara = new Person
val friends = List((alice, bob), (bob, clara))
clara.smokes.observe(true)

null

In [80]:
def smokingInfluence(pair: (Boolean, Boolean)) = 
if (pair._1 == pair._2) 3.0; else 1.0

smokingInfluence: (pair: (Boolean, Boolean))Double


In [81]:
for { (p1, p2) <- friends } { // for comprehension
    ^^(p1.smokes, p2.smokes).setConstraint(smokingInfluence)
}

Took a bit to figure this out, but here the model is the .smokes attribute for each person. You can apply a constraint outside of the declaration of the model. 

In [109]:
val alg = Importance(2000, alice.smokes)
alg.start()
println(alg.probability(alice.smokes, true))

0.7449926582847775


null

### Exploiting object oriented features for e.g. relational models

In [111]:
class Source(val name: String)

abstract class Sample {
    val fromSource: Element[Source]
}

class Pair(val source: Source, val sample: Sample) {
    val isRightSource = Apply(sample.fromSource,
                             (s: Source) => s == source)
    val distance = If(isRightSource,
                     Normal(0.0, 1.0),
                     Uniform(0.0, 10.0))
}

val source1 = new Source("Source 1")
val source2 = new Source("Source 2")
val sample1 = new Sample {
    val fromSource = Select(0.5 -> source1, 0.5 -> source2)
}
val pair1 = new Pair(source1, sample1)
val pair2 = new Pair(source2, sample1)

$line175.$read$$iw$$iw$$iw$$iw$$iw$$iw$Pair@68541931

In [112]:
pair1.distance.setCondition((d: Double) => (d > 0.15 && d < 0.25))
pair2.distance.setCondition((d: Double) => (d > 1.45 && d < 1.55))

In [128]:
// this breaks for any value of samples >= 20
val alg = Importance(10, pair2.isRightSource)
alg.start()
println(alg.probability(pair2.isRightSource, true))

0.5316237351004605


null

In [143]:
class Actor {
    val famous = Flip(0.1) 
}

class Movie {
    val quality = Select(0.3 -> 'low, 0.5 -> 'medium, 0.2 -> 'high)
}

defined class Actor
defined class Movie


In [146]:
import com.cra.figaro.library.compound.CPD
import com.cra.figaro.language._

import com.cra.figaro.library.compound.CPD
import com.cra.figaro.language._


In [144]:
class Appearance(actor: Actor, movie: Movie) {
    def probAward(quality: Symbol, famous: Boolean) = 
    (quality, famous) match { // matchus
        case ('low, false) => 0.001
        case('low, true) => 0.01
        case('medium, false) => 0.01
        case('medium, true) => 0.05
        case('high, false) => 0.05
        case('high, true) => 0.2
    }
    val award = Flip(Apply(movie.quality,
                        actor.famous,
                        (q: Symbol, f: Boolean) => probAward(q, f)))
}

// .....

// condition
def uniqueAward(awards: List[Boolean]) - 
awards.count((b: Boolean) => b) == 1

val allAwards: Element[List[Boolean]] = 
Inject(appearances.map(_.award):_*)

allAwards.setCondition(uniqueAward)

<console>: 19

### Using mutable fields can make capturing dependencies easier

In [150]:
import com.cra.figaro.library.collection.Container // ugh
class Actor {
    var movies: List[Movie] = List()
    lazy val skillful = Flip(0.1)
    lazy val qualities = Container(movies.map(_.quality):_*)
    lazy val numGoodMovies = qualities.count(_ == 'high)
    lazy val famous = Chain(numGoodMovies, (n: Int) => 
                           if (n >= 2) Flip(0.8) else Flip(0.1))
}

import com.cra.figaro.library.collection.Container
defined class Actor


In [151]:
class Movie {
    var actors: List[Actor] = List()
    lazy val skills = Container(actors.map(_.skillful):_*)
    lazy val actorsAllGood = skills.exists(b => b)
    lazy val probLow = 
    Apply(actorsAllGood, (b: Boolean) => if (b) 0.2; else 0.5)
    lazy val probHigh = 
    Apply(actorsAllGood, (b: Boolean) => if (b) 0.5; else 0.2)
    lazy val quality = 
    Select(probLow -> 'low, Constant(0.3) -> 'medium, probHigh -> 'high)
}

defined class Movie


In [155]:
class Appearance(actor: Actor, movie: Movie) {
    actor.movies ::= movie // this is like += for stuff like this
    movie.actors ::= actor
    
    def probAward(quality: Symbol, famous: Boolean) = 
    (quality, famous) match {
        case ('low, false) => 0.001
        case('low, true) => 0.01
        case('medium, false) => 0.01
        case('medium, true) => 0.05
        case('high, false) => 0.05
        case('high, true) => 0.2
    }
    lazy val award = Flip(Apply(movie.quality, actor.famous,
                               (q: Symbol, f: Boolean) => probAward(q, f)))
}

<console>: 18

In [12]:
val plot = new Plot
plot.add(new Line {x = (1.0 to 10.0 by 0.1); y = (1.0 to 10.0 by 0.1) map (sin (_))})